# 1. Data Preprocessing & Parquet
- Process the CSV file "marketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv"
- Cleanup & Create two Parquet files:
  - `products.parquet` (containing product `id`, `title`, `brand`, `category`, `price`, `rating`, `features`, and `ingredients`)

## Load CSV Data

- Load the 'marketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv' file into a pandas DataFrame.
- Inspect the column names to understand their structure before proceeding with extraction.

Please Ensure to use your own path to the csv file when running the file

In [1]:
import pandas as pd

# Modify to your path to CSV
file_path = '/content/drive/MyDrive/genAI final - personal/marketing_sample_for_amazon_com-ecommerce__20200101_20200131__10k_data.csv'

df = pd.read_csv(file_path)

print("First 5 rows of the DataFrame:")
print(df.head(3))
print("\nColumn names in the DataFrame:")
print(df.columns.tolist())

First 5 rows of the DataFrame:
                            Uniq Id  \
0  4c69b61db1fc16e7013b43fc926e502d   
1  66d49bbed043f5be260fa9f7fbff5957   
2  2c55cae269aebf53838484b0d7dd931a   

                                        Product Name  Brand Name  Asin  \
0  DB Longboards CoreFlex Crossbow 41" Bamboo Fib...         NaN   NaN   
1  Electronic Snap Circuits Mini Kits Classpack, ...         NaN   NaN   
2  3Doodler Create Flexy 3D Printing Filament Ref...         NaN   NaN   

                                            Category Upc Ean Code  List Price  \
0  Sports & Outdoors | Outdoor Recreation | Skate...          NaN         NaN   
1  Toys & Games | Learning & Education | Science ...          NaN         NaN   
2          Toys & Games | Arts & Crafts | Craft Kits          NaN         NaN   

  Selling Price  Quantity Model Number  ...  \
0       $237.68       NaN          NaN  ...   
1        $99.95       NaN        55324  ...   
2        $34.99       NaN          NaN  ...   

 

### Check for null columns

In [2]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10002 entries, 0 to 10001
Data columns (total 28 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Uniq Id                10002 non-null  object 
 1   Product Name           10002 non-null  object 
 2   Brand Name             0 non-null      float64
 3   Asin                   0 non-null      float64
 4   Category               9172 non-null   object 
 5   Upc Ean Code           34 non-null     object 
 6   List Price             0 non-null      float64
 7   Selling Price          9895 non-null   object 
 8   Quantity               0 non-null      float64
 9   Model Number           8230 non-null   object 
 10  About Product          9729 non-null   object 
 11  Product Specification  8370 non-null   object 
 12  Technical Details      9212 non-null   object 
 13  Shipping Weight        8864 non-null   object 
 14  Product Dimensions     479 non-null    object 
 15  Im

## Filter to Toys/Games/Party/Crafts domain slice

Our team has decided to use Toys/Games/Party/Crafts domain slice, which is a dominant slice in the dataset (>8000 rows)
- Filter based on toy keywords to achieve the slice

In [3]:
# ============================================================
# Filter to a Toys/Games/Party/Crafts domain slice
#    (based on Category text keywords)
# ============================================================

toy_keywords = [
    "Toys", "Toy", "Game", "Games", "Puzzle", "Jigsaw", "Figure", "Figures",
    "Doll", "Dolls", "Party", "Stuffed", "Plush", "Costume", "Craft", "Crafts",
    "Kids", "Play", "Vehicle", "Learning"
]

pattern = "|".join(toy_keywords)

mask_toys = df["Category"].str.contains(pattern, case=False, na=False)
toys_df = df[mask_toys].copy()

print("Toys/Party/Craft slice size:", len(toys_df))

Toys/Party/Craft slice size: 8202


## Prepare Products Data

- Extract, Clean-up, and transform the relevant columns from the loaded CSV data to create a DataFrame for `products.parquet`.


In [4]:
import numpy as np

# Initialize products_df
products_df = pd.DataFrame({
    'id': toys_df['Uniq Id'],
    'title': toys_df['Product Name'],
    'brand': toys_df['Brand Name'],
    'category': toys_df['Category']
})

# Clean and convert 'Selling Price' to numeric for 'price' column
products_df['price'] = toys_df['Selling Price'].astype(str).str.replace('$', '', regex=False).str.replace(',', '', regex=False)
products_df['price'] = pd.to_numeric(products_df['price'], errors='coerce')

# Initialize 'rating' column with NaN
products_df['rating'] = np.nan
# Fill the rating with random rating (numeric between 1-5)
products_df['rating'] = np.random.randint(1, 6, size=len(products_df))

# Concatenate text from relevant columns (not null) for 'features'
feature_columns = [
    'About Product',
    'Product Specification',
    'Technical Details'
]

# Fill NaN values with empty strings before concatenation
products_df['features'] = toys_df[feature_columns].fillna('').agg(' '.join, axis=1)

# Populate 'ingredients' column (but Ingredients column is all null)
products_df['ingredients'] = toys_df['Ingredients']

# Display the first few rows of products_df
print("\nFirst 5 rows of products_df:")
print(products_df.head())

# Check data types and non-null counts for products_df
print("\nInfo for products_df:")
products_df.info()


First 5 rows of products_df:
                                 id  \
1  66d49bbed043f5be260fa9f7fbff5957   
2  2c55cae269aebf53838484b0d7dd931a   
3  18018b6bc416dab347b1b7db79994afa   
4  e04b990e95bf73bbe6a3fa09785d7cd0   
6  40d3cd16b41970ae6872e914aecf2c8e   

                                               title  brand  \
1  Electronic Snap Circuits Mini Kits Classpack, ...    NaN   
2  3Doodler Create Flexy 3D Printing Filament Ref...    NaN   
3  Guillow Airplane Design Studio with Travel Cas...    NaN   
4                   Woodstock- Collage 500 pc Puzzle    NaN   
6  Rubie's Child's Pokemon Deluxe Pikachu Costume...    NaN   

                                            category  price  rating  \
1  Toys & Games | Learning & Education | Science ...  99.95       3   
2          Toys & Games | Arts & Crafts | Craft Kits  34.99       1   
3  Toys & Games | Hobbies | Models & Model Kits |...  28.91       3   
4            Toys & Games | Puzzles | Jigsaw Puzzles  17.49       1   
6

In [5]:
products_df.to_csv('products_df.csv', index=False)

## Fill in Ingredients using Meta-Llama-3-8B-Instruct (keyword extraction from features column)

Since ingredients are NaN in all rows, we will extract ingredients from features if available. If not, column value is filled as "Unknown"

Here is one example:

In [25]:
products_df.iloc[1]['features']

"Make sure this fits by entering your model number. | ✅【Smooth 3D drawing experienced the best 3D drawing experience by only using 3Doodler Create Plastics with 3Doodler Create+ and create 3D Printing pen. | ✅【Safe to use】the 3Doodler Create Plastics, conforms to the health requirements of ASTM-D-4236 & require no additional labelling in accordance with the US Consumer Product safety Commission’s Regulations as mandated by Labeling of Hazardous Art Materials Act (LHAMA). | 👍【3Doodler very own Type of plastic】the FLEXY plastic takes creativity to new levels! You can make flexible fun 3D creations! | 🌍【Environmentally friendly】3doodler create Plastic is made of corn and are 100% compostable! | ✅【125 strands of drawing fun】this bundle includes 5 refill filament packs, that's a total of 1043 ft. Of 3D drawing and doodling fun! | 📱【The 3Doodler app】get an interactive experience! The app is packed with dedicated easy to follow stencil section and step by step interactive instructions, receiv

"Plastic is made of corn and are 100% compostable"
We are expecting Llama to extract "corn, plastic" in this case.


In [6]:
!pip install transformers accelerate sentencepiece --quiet


In [3]:
# login to huggingface
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3-8B-Instruct",
    device_map="auto",
    torch_dtype=torch.float16
)

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3-8B-Instruct")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

...

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

In [15]:
sample_df = products_df.copy()

In [16]:

# Build context for each row
def build_product_context(row):
    parts = []
    for col in ["title", "features"]:
        if col in sample_df.columns:
            val = row.get(col)
        else:
            val = None
        if isinstance(val, str) and val.strip():
            parts.append(val.strip())
    return "\n".join(parts)


# LLM inference with HF Transformers

def generate_ingredient_keywords(text: str) -> str:
    if not isinstance(text, str) or not text.strip():
        return "unknown"

    system_prompt = (
        "You are an e-commerce data annotator.\n"
        "Given the product name and description, infer the likely main ingredients "
        "or material-related keywords of the product.\n"
        "- Answer ONLY in concise English.\n"
        "- Output a comma-separated list (e.g., 'aloe vera, glycerin, water').\n"
        "- If it is not a consumable, cosmetic, household chemical, pet/baby product, "
        "or you cannot infer ingredients, respond with 'unknown'."
    )

    user_prompt = f"Product information:\n{text}\n\nInferred ingredients or material keywords:"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    # LLaMA-3 chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=64,
            temperature=0.1,
            do_sample=False,
        )

    # get just the response
    gen_tokens = output[0][inputs["input_ids"].shape[1]:]
    answer = tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()

    # just the first row
    answer = answer.split("\n")[0].strip()

    lower = answer.lower()
    if "unknown" in lower or "not possible to infer" in lower:
        return "unknown"

    # Clean up patterns like "The answer is: polyester"
    for sep in ["the answer is", "answer:", "answer is"]:
        if sep in lower:
            idx = lower.find(sep)
            answer = answer[idx + len(sep):].strip(" :.").strip()
            break

    return answer


# Filter target rows with features
mask = sample_df["features"].notna()
target_df = sample_df[mask]

results = []

print(f"Target rows: {len(target_df)}")


# Run inference
# Start with 20 rows for prompt tuning
for i, (idx, row) in enumerate(target_df.iterrows(), start=1):
    ctx = build_product_context(row)
    ing_keywords = generate_ingredient_keywords(ctx)
    results.append((idx, ing_keywords))

    if i % 20 == 0:  # progress indicator
        print(f"Processed {i} rows.")


# Fill ingredients column
for idx, ing_keywords in results:
    sample_df.loc[idx, "ingredients"] = ing_keywords


# Save result
sample_df.to_csv("products_with_ingredients_generated.csv", index=False)
print("Saved: products_with_ingredients_generated.csv")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Target rows: 8202


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Processed 20 rows.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

...

Processed 8180 rows.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Processed 8200 rows.


/tmp/ipython-input-3872435276.py:97: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'unknown' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sample_df.loc[idx, "ingredients"] = ing_keywords


Saved: products_with_ingredients_generated.csv


In [2]:
import pandas as pd

sample_df = pd.read_csv('products_with_ingredients_generated.csv')

Check that ingredients field is correctly populated

In [22]:
sample_df.loc[0:5, 'ingredients']

,ingredients
1,unknown
2,"corn, plastic"
3,"balsa wood, plastic, rubber"
4,unknown


We also see "corn, plastic" correctly populated in the example we mentioned above.

## Check the info on clean up df again



In [28]:
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8202 entries, 1 to 10001
Data columns (total 8 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   id           8202 non-null   object 
 1   title        8202 non-null   object 
 2   brand        0 non-null      float64
 3   category     8202 non-null   object 
 4   price        7864 non-null   float64
 5   rating       8202 non-null   int64  
 6   features     8202 non-null   object 
 7   ingredients  8202 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 834.7+ KB


## Fill in the "Brands" Column using Llama

In [5]:
# Build context for each row
def build_product_context(row):
    parts = []
    for col in ["title", "features"]:
        if col in sample_df.columns:
            val = row.get(col)
        else:
            val = None
        if isinstance(val, str) and val.strip():
            parts.append(val.strip())
    return "\n".join(parts)


# LLM inference with HF Transformers

def generate_brand_name(text: str) -> str:
    if not isinstance(text, str) or not text.strip():
        return "unknown"

    system_prompt = (
        "You are an e-commerce data annotator.\n"
        "Given the product name and description, infer the most likely BRAND NAME "
        "or MANUFACTURER NAME of the product.\n"
        "- Answer ONLY with the brand/manufacturer name in concise English.\n"
        "- Do NOT include any extra words such as 'Brand:' or full sentences.\n"
        "- If you cannot infer a clear brand or manufacturer, respond with 'unknown'."
    )

    user_prompt = f"Product information:\n{text}\n\nInferred brand or manufacturer name:"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    # LLaMA-3 chat template
    prompt = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True,
    )

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=32,
            temperature=0.1,
            do_sample=False,
        )

    # get just the response
    gen_tokens = output[0][inputs["input_ids"].shape[1]:]
    answer = tokenizer.decode(gen_tokens, skip_special_tokens=True).strip()

    # just the first line
    answer = answer.split("\n")[0].strip()

    lower = answer.lower()
    if "unknown" in lower or "not possible to infer" in lower:
        return "unknown"

    # Clean up patterns like "The answer is: Disney"
    for sep in ["the answer is", "answer:", "answer is", "brand:", "brand is"]:
        if sep in lower:
            idx = lower.find(sep)
            answer = answer[idx + len(sep):].strip(" :.").strip()
            break

    return answer


# Filter target rows with features / title / description
mask = (
    sample_df.get("title", "").notna()
    | sample_df.get("features", "").notna()
)
target_df = sample_df[mask]

results = []

print(f"Target rows: {len(target_df)}")


# Run inference
# Start with 20 rows for prompt tuning
for i, (idx, row) in enumerate(target_df.iterrows(), start=1):
    ctx = build_product_context(row)
    brand_name = generate_brand_name(ctx)
    results.append((idx, brand_name))

    if i % 20 == 0:  # progress indicator
        print(f"Processed {i} rows.")


# Fill brand_generated column
if "brand" not in sample_df.columns:
    sample_df["brand"] = None

for idx, brand_name in results:
    sample_df.loc[idx, "brand"] = brand_name


# Save result
sample_df.to_csv("products_with_brand_generated.csv", index=False)
print("Saved: products_with_brand_generated.csv")

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Target rows: 8202


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Processed 20 rows.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

...

Processed 8180 rows.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for

Processed 8200 rows.


/tmp/ipython-input-589572061.py:102: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Elenco' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  sample_df.loc[idx, "brand"] = brand_name


Saved: products_with_brand_generated.csv


In [7]:
print("First 5 rows of products_df:")
print(sample_df['brand'].head(5))

First 5 rows of products_df:
0       Elenco
1     3Doodler
2      Guillow
3    Woodstock
4      Rubie's
Name: brand, dtype: object


## Clean-up "Features"

In [13]:
sample_df['features'].head(5)

,features
0,Snap circuits mini kits classpack provides bas...
1,Smooth 3D drawing experienced the best 3D draw...
2,Make 8 different Planes at one time. | Experim...
3,Puzzle has 500 pieces | Completed puzzle measu...
4,Polyester | Deluxe Pokemon child's Pikachu cos...


In [10]:
import re

def clean_features(text: str) -> str:
    if not isinstance(text, str):
        return text

    # 1. Remove the repeated default Amazon phrase
    prefix = "Make sure this fits by entering your model number. | "
    if text.startswith(prefix):
        text = text[len(prefix):]

    # 2. Remove emojis / non-ASCII characters
    # Keep only ASCII characters
    text = text.encode("ascii", "ignore").decode()

    # 3. Strip whitespace
    text = text.strip()

    # 4. Remove double spaces
    text = re.sub(r"\s+", " ", text)

    return text if text else None


# Apply to the whole column
sample_df["features"] = sample_df["features"].apply(clean_features)

print("Done cleaning!")
sample_df["features"].head()

Done cleaning!


,features
0,Snap circuits mini kits classpack provides bas...
1,Smooth 3D drawing experienced the best 3D draw...
2,Make 8 different Planes at one time. | Experim...
3,Puzzle has 500 pieces | Completed puzzle measu...
4,Polyester | Deluxe Pokemon child's Pikachu cos...


## Merge Rating fetched from Google Shopping API
Our group was able to fetch first 1200 rows' rating from the API, but was limited due to API # of request restriction on the free account.

We will merge these 1200 results, but randomly generate the rating for the rest of the products

In [15]:
rating_df = pd.read_csv('/content/Amazon_Product_Dataset_with_Rating.csv')

In [16]:
rating_df.head(5)

,id,title,brand,category,selling_price,list_price,search_text,rating,rating_count
0,66d49bbed043f5be260fa9f7fbff5957,"Electronic Snap Circuits Mini Kits Classpack, ...",NaN,Toys & Games | Learning & Education | Science ...,99.95,NaN,"Electronic Snap Circuits Mini Kits Classpack, ...",4.9,36.0
1,2c55cae269aebf53838484b0d7dd931a,3Doodler Create Flexy 3D Printing Filament Ref...,NaN,Toys & Games | Arts & Crafts | Craft Kits,34.99,NaN,3Doodler Create Flexy 3D Printing Filament Ref...,5.0,6.0
2,18018b6bc416dab347b1b7db79994afa,Guillow Airplane Design Studio with Travel Cas...,NaN,Toys & Games | Hobbies | Models & Model Kits |...,28.91,NaN,Guillow Airplane Design Studio with Travel Cas...,5.0,2.0
3,e04b990e95bf73bbe6a3fa09785d7cd0,Woodstock- Collage 500 pc Puzzle,NaN,Toys & Games | Puzzles | Jigsaw Puzzles,17.49,NaN,Woodstock- Collage 500 pc Puzzle Make sure thi...,5.0,1.0
4,40d3cd16b41970ae6872e914aecf2c8e,Rubie's Child's Pokemon Deluxe Pikachu Costume...,NaN,"Clothing, Shoes & Jewelry | Costumes & Accesso...",29.12,NaN,Rubie's Child's Pokemon Deluxe Pikachu Costume...,4.5,17.0


In [19]:
# print number of nan in rating_df["rating"]
print(rating_df["rating"].isna().sum())

6782


In [6]:
import numpy as np

# 1) Merge rating_df ratings into cleaned_df
merged = sample_df.merge(
    rating_df[["id", "rating"]],
    on="id",
    how="left",
    suffixes=("", "_real")
)

# 2) Fill NaN rating with random numbers
missing_mask = merged["rating_real"].isna()
merged.loc[missing_mask, "rating_real"] = np.round(np.random.uniform(1.0, 5.0, size=missing_mask.sum()), 1)

# 3) Ensure final rating is float
merged["rating"] = merged["rating_real"].astype(float)

# Drop helper column
merged = merged.drop(columns=["rating_real"])

merged.head()

,id,title,brand,category,price,rating,features,ingredients
0,66d49bbed043f5be260fa9f7fbff5957,"Electronic Snap Circuits Mini Kits Classpack, ...",Elenco,Toys & Games | Learning & Education | Science ...,99.95,4.9,Snap circuits mini kits classpack provides bas...,unknown
1,2c55cae269aebf53838484b0d7dd931a,3Doodler Create Flexy 3D Printing Filament Ref...,3Doodler,Toys & Games | Arts & Crafts | Craft Kits,34.99,5.0,Smooth 3D drawing experienced the best 3D draw...,"corn, plastic"
2,18018b6bc416dab347b1b7db79994afa,Guillow Airplane Design Studio with Travel Cas...,Guillow,Toys & Games | Hobbies | Models & Model Kits |...,28.91,5.0,Make 8 different Planes at one time. | Experim...,"balsa wood, plastic, rubber"
3,e04b990e95bf73bbe6a3fa09785d7cd0,Woodstock- Collage 500 pc Puzzle,Woodstock,Toys & Games | Puzzles | Jigsaw Puzzles,17.49,5.0,Puzzle has 500 pieces | Completed puzzle measu...,unknown
4,40d3cd16b41970ae6872e914aecf2c8e,Rubie's Child's Pokemon Deluxe Pikachu Costume...,Rubie's,"Clothing, Shoes & Jewelry | Costumes & Accesso...",29.12,4.5,Polyester | Deluxe Pokemon child's Pikachu cos...,Polyester


## Final Review of the Cleaned Data and Export as CSV

In [9]:
merged.head(5)

,id,title,brand,category,price,rating,features,ingredients
0,66d49bbed043f5be260fa9f7fbff5957,"Electronic Snap Circuits Mini Kits Classpack, ...",Elenco,Toys & Games | Learning & Education | Science ...,99.95,4.9,Snap circuits mini kits classpack provides bas...,unknown
1,2c55cae269aebf53838484b0d7dd931a,3Doodler Create Flexy 3D Printing Filament Ref...,3Doodler,Toys & Games | Arts & Crafts | Craft Kits,34.99,5.0,Smooth 3D drawing experienced the best 3D draw...,"corn, plastic"
2,18018b6bc416dab347b1b7db79994afa,Guillow Airplane Design Studio with Travel Cas...,Guillow,Toys & Games | Hobbies | Models & Model Kits |...,28.91,5.0,Make 8 different Planes at one time. | Experim...,"balsa wood, plastic, rubber"
3,e04b990e95bf73bbe6a3fa09785d7cd0,Woodstock- Collage 500 pc Puzzle,Woodstock,Toys & Games | Puzzles | Jigsaw Puzzles,17.49,5.0,Puzzle has 500 pieces | Completed puzzle measu...,unknown
4,40d3cd16b41970ae6872e914aecf2c8e,Rubie's Child's Pokemon Deluxe Pikachu Costume...,Rubie's,"Clothing, Shoes & Jewelry | Costumes & Accesso...",29.12,4.5,Polyester | Deluxe Pokemon child's Pikachu cos...,Polyester


In [ ]:
merged.to_csv('products_df_merged_cleaned.csv', index=False)

## Save Products Data to Parquet

Save the prepared products DataFrame to a Parquet file named `products.parquet`.


In [10]:
merged.to_parquet('products.parquet', index=False)
print("merged successfully saved to 'products.parquet'")

merged successfully saved to 'products.parquet'
